In [18]:
import os

SEASONS = [2024] #list(range(2015,2025))

DATA_DIR = "data"
STANDINGS_DIR = os.path.join(DATA_DIR, "standings")
SCORES_DIR = os.path.join(DATA_DIR, "scores")

In [19]:
from bs4 import BeautifulSoup
from playwright.async_api import async_playwright, TimeoutError as PlaywrightTimeout
import time
# Make sure to install playwright browsers by running playwright install on the command line or !playwright install from Jupyter

In [20]:
# Get html of each game
async def get_html(url, selector, sleep=10, retries=5):
    html = None
    for i in range(1, retries+1):
        time.sleep(sleep * i) # set sleep amount to reduce timeout occurences
        try:
            async with async_playwright() as p:
                browser = await p.chromium.launch()
                page = await browser.new_page()
                await page.goto(url)
                print(await page.title())
                html = await page.inner_html(selector)
        except PlaywrightTimeout:
            print(f"Timeout error on {url}")
            continue
        else:
            break
    return html

In [21]:
# Scraping games in a season
async def scrape_season(season):
    url = f"https://www.basketball-reference.com/leagues/NBA_{season}_games.html"
    html = await get_html(url, "#content .filter")

    soup = BeautifulSoup(html)
    links = soup.find_all("a")
    standings_pages = [f"https://www.basketball-reference.com{l['href']}" for l in links]

    # Go through each url in each standing (season)
    for url in standings_pages:
        save_path = os.path.join(STANDINGS_DIR, url.split("/")[-1])
        if os.path.exists(save_path):
            continue

        html = await get_html(url, "#all_schedule")
        with open(save_path, "w+") as f:
            f.write(html)

In [22]:
# Scraping seasons
for season in SEASONS:
    await scrape_season(season)

Timeout error on https://www.basketball-reference.com/leagues/NBA_2024_games.html
Timeout error on https://www.basketball-reference.com/leagues/NBA_2024_games.html
Timeout error on https://www.basketball-reference.com/leagues/NBA_2024_games.html
2023-24 NBA Schedule | Basketball-Reference.com
Timeout error on https://www.basketball-reference.com/leagues/NBA_2024_games-february.html
2023-24 NBA Schedule | Basketball-Reference.com


In [8]:
# Getting the files by months
standings_files = os.listdir(STANDINGS_DIR)

In [9]:
# Getting game statistics from each game
async def scrape_game(standings_file):
    with open(standings_file, 'r') as f:
        html = f.read()

    soup = BeautifulSoup(html)
    links = soup.find_all("a")
    hrefs = [l.get('href') for l in links]
    box_scores = [f"https://www.basketball-reference.com{l}" for l in hrefs if l and "boxscore" in l and '.html' in l]

    for url in box_scores:
        save_path = os.path.join(SCORES_DIR, url.split("/")[-1])
        if os.path.exists(save_path):
            continue

        html = await get_html(url, "#content")
        if not html:
            continue
        with open(save_path, "w+") as f:
            f.write(html)

In [23]:
import pandas as pd

for season in SEASONS:
    files = [s for s in standings_files if str(season) in s]

    for f in files:
        filepath = os.path.join(STANDINGS_DIR, f)

        await scrape_game(filepath) # scraping data from individual games

Clippers vs Pistons, February 2, 2024 | Basketball-Reference.com
Heat vs Wizards, February 2, 2024 | Basketball-Reference.com
Timeout error on https://www.basketball-reference.com/boxscores/202402020ATL.html
Suns vs Hawks, February 2, 2024 | Basketball-Reference.com
Timeout error on https://www.basketball-reference.com/boxscores/202402020IND.html
Timeout error on https://www.basketball-reference.com/boxscores/202402020IND.html
Kings vs Pacers, February 2, 2024 | Basketball-Reference.com
Raptors vs Rockets, February 2, 2024 | Basketball-Reference.com
Warriors vs Grizzlies, February 2, 2024 | Basketball-Reference.com
Magic vs Timberwolves, February 2, 2024 | Basketball-Reference.com
Hornets vs Thunder, February 2, 2024 | Basketball-Reference.com
Pelicans vs Spurs, February 2, 2024 | Basketball-Reference.com
Trail Blazers vs Nuggets, February 2, 2024 | Basketball-Reference.com
Nets vs 76ers, February 3, 2024 | Basketball-Reference.com
Warriors vs Hawks, February 3, 2024 | Basketball-Refer